<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


발사 성공률은 페이로드 질량, 궤도 유형 등과 같은 여러 요소에 따라 달라질 수 있습니다. 또한 발사 지점의 위치와 근접성, 즉 로켓 궤적의 초기 위치에 따라 달라질 수도 있습니다. 발사장 건설을 위한 최적의 위치를 찾는 것은 확실히 많은 요소를 포함하며, 바라건대 기존 발사장 위치를 분석하여 일부 요소를 발견할 수 있기를 바랍니다.

이전 탐색적 데이터 분석 랩에서는 matplotlib 및 seaborn을 사용하여 SpaceX 발사 데이터 세트를 시각화하고 발사 사이트와 성공률 간의 몇 가지 예비 상관 관계를 발견했습니다. 이 실습에서는 Folium을 사용하여 더욱 대화형의 시각적 분석을 수행하게 됩니다.

## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
!pip3 install folium
!pip3 install wget

## Import

In [1]:
import folium
import wget
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [ ]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [3]:
# 이번에도 따로

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(url)
spacex_df.head(3)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [ ]:
# 4개 컬럼만 남기는거군.

In [9]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
spacex_df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [ ]:
# Launch Site 별로 위도, 경고 값이 중복되어 있어서 1개만 남겨서 중복 제거.
# Class 컬럼도 제거하네.

In [10]:
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


G

1. `spacex_df`에서 필요한 열 선택:
   - `spacex_df[['Launch Site', 'Lat', 'Long', 'class']]`은 `spacex_df` 데이터프레임에서 'Launch Site', 'Lat', 'Long', 및 'class' 열을 선택합니다. 선택한 열들로 새로운 데이터프레임을 생성하려는 것입니다.

2. `spacex_df`를 'Launch Site' 열을 기준으로 그룹화:
   - `spacex_df.groupby(['Launch Site'], as_index=False)`은 'Launch Site' 열을 기준으로 데이터를 그룹화합니다. `as_index=False` 옵션을 사용하여 그룹화된 결과에서 'Launch Site' 열을 인덱스로 사용하지 않도록 설정합니다.

3. 그룹화된 그룹 중 첫 번째 행 선택:
   - `first()` 함수를 사용하여 각 그룹 내에서 첫 번째 행을 선택합니다. 이것은 각 발사 사이트에 대한 정보 중 하나를 선택하고 중복을 제거합니다.

In [8]:
# test
spacex_df.groupby(['Launch Site'], as_index=False).first()


,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


위의 좌표는 발사 지점이 어디에 있는지 직관적인 통찰력을 제공할 수 없는 단순한 숫자일 뿐입니다. 지리학에 능숙하다면 그 숫자를 마음속으로 직접 해석할 수 있습니다. 그렇지 않다면 그것도 괜찮습니다. 해당 위치를 지도에 고정하여 시각화해 보겠습니다.

먼저 초기 중심 위치를 텍사스 주 휴스턴에 있는 NASA 존슨 우주 센터로 설정하여 Folium Map 개체를 만들어야 합니다.

In [11]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True
                      ).add_child(folium.Popup('NASA Johnson Space Center'))

In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(  icon_size=(20,20)
                 , icon_anchor=(0,0)
                 , html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
                )
    )

In [15]:
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [42]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. 
# In addition, add Launch site name as a popup label


In [43]:
# for lat, long in launch_sites_df[['Lat','Long']]:
for index, row in launch_sites_df.iterrows():
    lat = row['Lat']
    long = row['Long']
    site = row['Launch Site']
    print(f'[debug] site:{site}, lat:{lat}, long:{long}')

    # Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
    circle = folium.Circle([lat,long], radius=1000, color='#d35400', fill=True
                          ).add_child(folium.Popup(site))
    # Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
        [lat,long],
        # Create an icon as a text label
        icon=DivIcon(  icon_size=(20,20)
                     , icon_anchor=(0,0)
                     , html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site,
                    )
        )
    
    site_map.add_child(circle)
    site_map.add_child(marker)
    
site_map

[debug] site:CCAFS LC-40, lat:28.56230197, long:-80.57735648
[debug] site:CCAFS SLC-40, lat:28.56319718, long:-80.57682003
[debug] site:KSC LC-39A, lat:28.57325457, long:-80.64689529
[debug] site:VAFB SLC-4E, lat:34.63283416, long:-120.6107455


In [16]:
# test
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


이제 표시된 영역을 확대/축소하여 지도를 탐색하고 다음 질문에 답해 보세요.

- 모든 발사 지점이 Equator 선 근처에 있습니까?
- 모든 발사 장소가 해안과 매우 가까운 곳에 있습니까?

또한 연구 결과를 설명해주세요.

## Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [44]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


다음으로 모든 실행 기록에 대한 마커를 만들어 보겠습니다. 실행이 성공하면(클래스=1) 녹색 마커를 사용하고, 실행에 실패하면 빨간색 마커(클래스=0)를 사용합니다.

발사는 4개의 발사 장소 중 한 곳에서만 발생하므로 많은 발사 기록이 정확히 동일한 좌표를 갖게 됩니다. 마커 클러스터는 동일한 좌표를 가진 많은 마커가 포함된 지도를 단순화하는 좋은 방법이 될 수 있습니다.

Let's first create a `MarkerCluster` object


In [45]:
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [46]:
# test
spacex_df.head(3)

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0


In [ ]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [47]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


G

4. `spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)` 코드는 'class' 열의 각 행에 `assign_marker_color` 함수를 적용하여 'marker_color' 열에 해당하는 마커 색상을 할당합니다. 'class' 열은 SpaceX 발사의 성공(1) 또는 실패(0)를 나타내므로, 해당 결과에 따라 'marker_color' 열이 'green' 또는 'red'로 채워집니다.

이렇게 함으로써 데이터프레임에 새로운 열이 추가되고, 각 행의 'class' 값에 따라 해당하는 색상이 'marker_color' 열에 저장됩니다. 이러한 색상 정보는 지도에 마커를 그릴 때 사용됩니다.

_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [48]:
# test
spacex_df.head(2)

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red


In [57]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    
    
    lat = record['Lat']
    long = record['Long']
    site = record['Launch Site']
    color= record['marker_color']
    # print(f'[debug] site:{site}, lat:{lat}, long:{long}, color:{color}')

    # Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
    #circle = folium.Circle([lat,long], radius=1000, color=color, fill=True
    circle = folium.Circle([lat,long], radius=10, color=color, fill=True
                          ).add_child(folium.Popup(site))
    
    # Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
        [lat,long],
        # Create an icon as a text label
        icon=DivIcon(  icon_size=(20,20)
                     , icon_anchor=(0,0)
                     , html='<div style="font-size: 12; color:color;"><b>%s</b></div>' % site,
                    )
        )
    marker_cluster.add_child(circle)
    
    
    marker_cluster.add_child(marker)

site_map

- 도대체 답이 뭐야?
- what is the answer?

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


## TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [61]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [62]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [63]:
# test
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [65]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [65]:
coastline_lat = 28.56367
coastline_lon = -80.57163

for index, row in launch_sites_df.iterrows():
    distance_coastline = calculate_distance(row['Lat'], row['Long'], coastline_lat, coastline_lon)
    print(f'distance_coastline:{distance_coastline}')

distance_coastline:0.5797581813109574
distance_coastline:0.5097439631188213
distance_coastline:7.429343134068913
distance_coastline:3827.511985578491


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


- 아, 재미 없어! (Oh no fun!)

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [66]:
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
       )
   )

NameError: name 'coordinate' is not defined

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [67]:
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

NameError: name 'coordinates' is not defined

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


## Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
